In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from statsmodels.tsa.vector_ar.var_model import VAR

from sklearn import preprocessing
from sklearn.decomposition import PCA

import warnings
warnings.simplefilter('ignore')

In [ ]:
df = pd.read_csv("kpis_cleaned_notonehot_median.csv")
drop_columns = ['tech', 'freq', 'site', 'sector', 'Unnamed: 0']
df.drop(columns=drop_columns, inplace=True)
df.head()

,cell_name,timestamp,ho_failure_rate,num_voice_attempts,voice_drop_rate,num_data_attempts,voice_setup_failure_rate,voice_tot_failure_rate,avail_period_duration,bandwidth,throughput_rate,data_setup_failure_rate,data_drop_rate,data_tot_failure_rate,unavail_total_rate,unavail_unplan_rate,voice_setup_failure_rate_is_nan,voice_tot_failure_rate_is_nan,data_setup_failure_rate_is_nan,data_tot_failure_rate_is_nan,tech_freq
0,00_11Z,2020-04-09 15:00:00+00:00,0.333333,0.000927,0.000000,0.004527,0.000000,0.000000,1.0,0.49975,0.000195,0.000000,0.000731,0.000731,0.333364,0.0,False,False,False,False,Z
1,00_11Z,2020-04-22 14:00:00+00:00,0.360000,0.017609,0.000000,0.012312,0.000000,0.000000,1.0,0.49975,0.000197,0.001075,0.000269,0.000403,0.333364,0.0,False,False,False,False,Z
2,00_11Z,2020-05-08 21:00:00+00:00,0.333333,0.002780,0.000000,0.008115,0.000000,0.000000,1.0,0.49975,0.000196,0.000000,0.000000,0.000000,0.333364,0.0,False,False,False,False,Z
3,00_11Z,2020-05-10 13:00:00+00:00,0.380952,0.012048,0.000000,0.004898,0.000000,0.000000,1.0,0.49975,0.000116,0.000000,0.002027,0.002027,0.333364,0.0,False,False,False,False,Z
4,00_11Z,2020-05-12 03:00:00+00:00,0.002273,0.002273,0.002273,0.002273,0.002273,0.002273,1.0,0.49975,0.002273,0.002273,0.002273,0.002273,0.666728,0.0,True,False,False,False,Z


In [ ]:
df = pd.get_dummies(df, columns=['tech_freq'])*1
df.head()

,cell_name,timestamp,ho_failure_rate,num_voice_attempts,voice_drop_rate,num_data_attempts,voice_setup_failure_rate,voice_tot_failure_rate,avail_period_duration,bandwidth,throughput_rate,data_setup_failure_rate,data_drop_rate,data_tot_failure_rate,unavail_total_rate,unavail_unplan_rate,voice_setup_failure_rate_is_nan,voice_tot_failure_rate_is_nan,data_setup_failure_rate_is_nan,data_tot_failure_rate_is_nan,tech_freq_P,tech_freq_Q,tech_freq_R,tech_freq_V,tech_freq_W,tech_freq_X,tech_freq_Y,tech_freq_Z
0,00_11Z,2020-04-09 15:00:00+00:00,0.333333,0.000927,0.000000,0.004527,0.000000,0.000000,1.0,0.49975,0.000195,0.000000,0.000731,0.000731,0.333364,0.0,0,0,0,0,0,0,0,0,0,0,0,1
1,00_11Z,2020-04-22 14:00:00+00:00,0.360000,0.017609,0.000000,0.012312,0.000000,0.000000,1.0,0.49975,0.000197,0.001075,0.000269,0.000403,0.333364,0.0,0,0,0,0,0,0,0,0,0,0,0,1
2,00_11Z,2020-05-08 21:00:00+00:00,0.333333,0.002780,0.000000,0.008115,0.000000,0.000000,1.0,0.49975,0.000196,0.000000,0.000000,0.000000,0.333364,0.0,0,0,0,0,0,0,0,0,0,0,0,1
3,00_11Z,2020-05-10 13:00:00+00:00,0.380952,0.012048,0.000000,0.004898,0.000000,0.000000,1.0,0.49975,0.000116,0.000000,0.002027,0.002027,0.333364,0.0,0,0,0,0,0,0,0,0,0,0,0,1
4,00_11Z,2020-05-12 03:00:00+00:00,0.002273,0.002273,0.002273,0.002273,0.002273,0.002273,1.0,0.49975,0.002273,0.002273,0.002273,0.002273,0.666728,0.0,1,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
def preprocess(df):
  df_temp = df.fillna(df.median(), axis='index')
  df_temp = df_temp.drop(columns='cell_name')
  df_temp = df_temp.sort_values('timestamp')
  return df_temp.set_index('timestamp')

In [ ]:
df = preprocess(df)

In [ ]:
df.to_csv("ml_models_dataset.csv")

In [ ]:
def create_VAR_model(data, cell_name, n_pca):
  # PCA
  data = data[data.cell_name == cell_name]
  data.drop(labels="cell_name", axis=1, inplace=True)

  pca = PCA(n_components=n_pca)
  pca.fit(data)

  columns = [f'pca_{i}' for i in range(n_pca)]
  df_pca = pd.DataFrame(pca.transform(data), columns=columns, index=data.index)

  # Model Selection
  AIC = {}
  best_aic, best_order = np.inf, 0

  for i in range(1, 50):
      model = VAR(endog=df_pca)
      var_result = model.fit(maxlags=i, trend="nc")
      AIC[i] = var_result.aic
      
      if AIC[i] < best_aic:
          best_aic = AIC[i]
          best_order = i

  # Best model
  var = VAR(endog=df_pca)
  var_result = var.fit(maxlags=best_order)

  return var_result, df_pca, best_order

In [ ]:
def find_anomalies(data, cell_name, n_pca):
  var_result, df_pca, best_order = create_VAR_model(data, cell_name, n_pca)

  residuals_mean = var_result.resid.values.mean(axis=0)
  residuals_std = var_result.resid.values.std(axis=0)

  residuals = (var_result.resid.values - residuals_mean) / residuals_std
  cov_residuals = np.linalg.inv(np.cov(residuals.T))

  T = np.diag((residuals).dot(cov_residuals).dot(residuals.T))

  m = var_result.nobs
  p = var_result.resid.shape[-1]
  alpha = 0.01

  UCL = stats.f.ppf(1-alpha, dfn=p, dfd=m-p) *(p*(m+1)*(m-1)/(m*m-m*p))

  anomaly_timestamps = []
  for i, t in enumerate(T):
    if t > UCL:
      anomaly_timestamps.append(list(df_pca.index[best_order:])[i])
  
  return anomaly_timestamps